[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# *Pydantic* y *SQLAlchemy*.

Es posible relacionar un esquema definido con *Pydantic* a un modelo creado con *SQLAlchemy*.

## El atributo   ```Config``` de ```pydantic.BaseModel```.

Este atributo se define como una clase y contiene metadatos que sirven para configurar a las subclases de ```pydantic.BaseModel```. 

Uno de atributos importantes de ```Config``` es ```orm_mode```, el cual, cuando su valor es ```Tue```, indica que los objetos instanciados pueden interactuar con un ORM.

## El atributo ```from_orm()```.

Este atributo permite crear objetos instanciados de subclases de ```BaseModel``` ligados a un modelo de *ORM*.

## Ejemplo ilustrativo.

In [ ]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from pydantic import BaseModel, validator
from typing import Literal

### Creación del modelo ```Alumno``` con *SQLAlchemy*.

In [ ]:
Base = declarative_base()

In [ ]:
class Alumno(Base):
    __tablename__ = 'alumnos'
    cuenta = sa.Column(sa.Integer, primary_key=True)
    nombre = sa.Column(sa.String(50))
    primer_apellido = sa.Column(sa.String(50))
    segundo_apellido = sa.Column(sa.String(50))
    carrera = sa.Column(sa.String(50))
    semestre = sa.Column(sa.Integer)
    promedio = sa.Column(sa.Float)
    al_corriente = sa.Column(sa.Boolean)

### Inicio de la sesión.

In [ ]:
engine = sa.create_engine('sqlite:///db.sqlite3')

In [ ]:
Session = sessionmaker(bind=engine)

In [ ]:
session = Session()

In [ ]:
Base.metadata.create_all(engine)

### Creación del esquema ```AlumnoSchema``` con *Pydantic*.

In [ ]:
CARRERAS = ['Arquitectura',
            'Diseño',
            'Sistemas',
            'Derecho',
            'Actuaría']

In [ ]:
class AlumnoSchema(BaseModel):
    cuenta: int
    nombre: str
    primer_apellido: str
    segundo_apellido: str = ''
    carrera: Literal[tuple(CARRERAS)]
    semestre: int
    promedio: float
    al_corriente: bool
        
    class Config:
        orm_mode = True

* La siguiente celda creará un objeto ```list``` con el nombre ```alumnos``` que contendrá las instancias de ```Alumno``` construidas a partir de todos los registros de la tabla ```alumnos``` en la base de datos.

### Extracción del estado del modelo desde la base de datos.

In [ ]:
alumnos = session.query(Alumno).filter(Alumno.cuenta).all()

In [ ]:
alumnos

### Serialización del estado del modelo.

In [ ]:
datos_alumnos = [AlumnoSchema.from_orm(alumno) for alumno in alumnos]

In [ ]:
datos_alumnos

### Validación y adición de un nuevo registro a la base de datos.

In [ ]:
datos = {'cuenta': 1234567,
         'nombre': 'Juan',
         'primer_apellido': 'Pérez',
         'segundo_apellido': 'López',
         'carrera': 'Sistemas',
         'semestre': 7,
         'promedio': 6.5,
        'al_corriente': True}

In [ ]:
session.add(Alumno(**AlumnoSchema(**datos).dict()))

In [ ]:
session.commit()

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///db.sqlite3

In [ ]:
%sql select * from alumnos

### Eliminación de un registro de la base de datos.

In [ ]:
alumno_borrar = session.query(Alumno).filter_by(cuenta=1234567).first()
session.delete(alumno_borrar)
session.commit()

In [ ]:
%sql select * from alumnos

In [ ]:
session.close()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>